In [225]:
class Entry:
    def __init__(self, date, price):
        self.cost = price
        self.sl = price * 0.9
        self.date = date
    def update(self, current_price):
        if current_price >= self.cost * 1.10:
            return True
        
        if current_price < self.sl:
            return True
        else:
            #self.sl = max(current_price * .9, self.sl)
            return False

### Form dataframe, compute metrics for confidence level

In [221]:
import pandas as pd
import numpy as np

def stochastic(df: pd.DataFrame, num_days: int) -> pd.DataFrame:
    min_low = df['Low'].rolling(num_days).min()
    max_high = df['High'].rolling(num_days).max()
    return ((df['Close']-min_low)/(max_high-min_low)).rolling(3).mean().rolling(3).mean()

def support_levels(df: pd.DataFrame):
    pivot = df[['High', 'Low', 'Close']].shift(1).mean(axis=1)
    return pivot * 2 - df['High'].shift(1)

df = pd.read_csv('data/spy_price.csv')
df = df.join(pd.read_csv('data/spy_vs_25dr.csv').drop('Date', axis=1), )
df['Support'] = support_levels(df)
df['Stochastic'] = stochastic(df, 14)

df['Metric 1'] = df['Stochastic'] - 0.2
df['Metric 2'] = df['Stochastic'] - df['Stochastic'].rolling(5).mean()
df['Metric 3'] = df['Put - Call Diff'] - df['Put - Call Diff'].rolling(10).mean()
df['Metric 4'] = (df['Put - Call Diff'] - df['Put - Call Diff'].shift(1)).rolling(20).mean()

df['Confidence'] = df['Metric 1'] + df['Metric 2'] + df['Metric 3'] + df['Metric 4']
df.dropna(inplace=True)

df = df.loc[df['Date'].eq('2023-03-01').idxmax():]


 ### Run backtest

In [222]:
import datetime

entries: list[Entry] = []
total_entries, good_entries = 0, 0
delta = 0
entries_to_csv = pd.DataFrame(columns=['Entry Date', 'Exit Date', 'Entry', 'Exit', 'P/L'])
for date, close, confidence in zip(df['Date'], df['Close'], df['Confidence']):
    for entry in entries:
        if entry.update(close):
            mult = (close/entry.cost)
            total_entries += 1
            if mult > 1:
                good_entries += 1
            delta += 10 * (mult-1)
            entries.remove(entry)
            entries_to_csv.loc[len(entries_to_csv)] = [entry.date, date, entry.cost, close, close/entry.cost]
            
    if confidence < 0:
        entries.append(Entry(date, close))
closed_entries = total_entries

avg_pos_days = (pd.to_datetime(entries_to_csv['Exit Date']) - pd.to_datetime(entries_to_csv['Entry Date'])).median().days

# parse unclosed positions
for entry in entries:
    # entries_to_csv.loc[len(entries_to_csv)] = [entry.date, None, entry.cost, None, None]
    total_entries += 1
    
entries_to_csv.sort_values(by=['Entry Date'], axis=0, inplace=True)


### Display stats from backtest

In [223]:
if total_entries == 0:
    print('No entries made.')
else:
    print('Total Entries:', total_entries, f'[{good_entries}/{total_entries}] (%{100*good_entries/total_entries:.2f})')
    print('Closed Entries:', closed_entries, f'[{good_entries}/{closed_entries}] (%{100*good_entries/closed_entries:.2f})')
    money_in = 10 * total_entries
    money_out = money_in + delta
    print(f'WR: {100*good_entries/total_entries:.2f} [{good_entries}/{total_entries}]')
    print(f'Average P/L per entry: %{100*money_out/money_in-100:.2f}')
    print('Median time per entry:', avg_pos_days, 'days')
    print(f'Money in: ${money_in:.2f}')
    print(f'Money out: ${money_out:.2f}')
    
    
row_colors = np.where(entries_to_csv['P/L'] > 1, 'background-color: green', 'background-color: red')
styler = entries_to_csv.style.apply(lambda _: row_colors)
styler.to_excel('entries_made.xlsx', index=False)

print('\nEntries unclosed:')
for entry in entries:
    print(entry.date, f'${entry.cost:.2f}')

Total Entries: 65 [62/65] (%95.38)
Closed Entries: 62 [62/62] (%100.00)
WR: 95.38 [62/65]
Average P/L per entry: %10.07
Median time per entry: 100 days
Money in: $650.00
Money out: $715.47

Entries unclosed:
2023-12-19 $474.84
2024-03-19 $515.71
2024-03-20 $520.48


### (not updated) graphs

In [224]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, subplot_titles=['SPY Chart', 'Confidence'], shared_xaxes=True)

fig.add_candlestick(
    x = df['Date'],
    open = df['Open'],
    high = df['High'],
    low = df['Low'],
    close = df['Close'],
    name='SPY',
    row=1,
    col=1
)
fig.update_xaxes(rangeslider_visible=False)
fig.add_scatter(x=df['Date'], y=df['Confidence'], row=2, col=1)

fig.show()